# crvUSD market oracles: wstETH

The main function in terms of calculating the oracle price of the collateral is the internal function`_raw_price()` and `_ema_tvl()`.

Every market has its own individual price oracle contract, which can be fetched by calling `price_oracle_contract` within the controller of the market. The [wstETH oracle](https://etherscan.io/address/0xc1793A29609ffFF81f10139fa0A7A444c9e106Ad#code) will be used for the purpose of this documentation. Please be aware that oracle contracts can vary based on the collateral token.

In [69]:
# importing ape and math + fork ethereum mainnet

import ape
from ape import chain, networks
import math


# fork eth mainnet
ape.networks.parse_network_choice('ethereum:mainnet-fork').__enter__()

INFO: Starting 'Hardhat node' process.
INFO: Connecting to existing Geth node at https://eth-mainnet.g.alchemy.com/[hidden].
INFO: Stopping 'Hardhat node' process.


<hardhat chain_id=31337>


`_ema_tvl()` calculates the exponential moving average of the total value locked of `TRICRYPTO[i]`. 
This value is later on used in the internal function `_raw_price()` to further compute the weighted price of the collateral.
`_ema_tvl()` calculates the exponential moving average of the total value locked (TVL) for `TRICRYPTO[i]` and returns `last_tvl[i]`, which represents the Exponential Moving Average (EMA) of the TVL of the Tricrypto pool at index i. This variable is updated every time `price_w()` is called and $last_{timestamp} < block.timestamp$. If the latter condition is not met, it will simply return last_tvl.

This function returns `last_tvl[i]`, which represents the ema tvl of the pool. This variable is updated everytime when calling `price_w()` and $last_{timestamp} < block.timestamp$.
This value is subsequently used in the internal function `_raw_price()` to compute the weighted price of ETH.

The function only re-calculates the ema tvl when $last_{timestamp} < block.timestamp$, otherwise it will just return `last_tvl` again as it is still the same timestamp. 

In [70]:
# contract addresses 
price_oracle_contract = ape.Contract('0xc1793A29609ffFF81f10139fa0A7A444c9e106Ad')
TRICRYPTO = ["0x7F86Bf177Dd4F3494b841a37e810A34dD56c829B", "0xf5f5B97624542D72A9E06f04804Bf81baA15e2B4"]
STABLESWAP = ["0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E", "0x390f3595bCa2Df7d23783dFd126427CCeb997BF4"]
STAKEDSWAP = ape.Contract("0x21E27a5E5513D6e65C4f830167390997aA84843a")
WSTETH = ape.Contract("0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0")
stable_agg = ape.Contract("0x18672b1b0c623a30089A280Ed9256379fb0E4E62")

In [71]:
# aggregate needed variables for calculations
N_POOLS = price_oracle_contract.N_POOLS()
last_timestamp = price_oracle_contract.last_timestamp()
block_timestamp = chain.provider.get_block("latest").timestamp
TVL_MA_TIME = price_oracle_contract.TVL_MA_TIME()

# empty lists to store tvls in 
last_tvl = []
last_tvl_new = []

## **EMA of TVL**

`_ema_tvl()` calculates the exponential moving average of the total value locked of `TRICRYPTO[i]`. 
This value is later on used in the internal function `_raw_price()` to further compute the weighted price of the collateral.
`_ema_tvl()` calculates the exponential moving average of the total value locked (TVL) for `TRICRYPTO[i]` and returns `last_tvl[i]`, which represents the Exponential Moving Average (EMA) of the TVL of the Tricrypto pool at index i. This variable is updated every time `price_w()` is called and $last_{timestamp} < block.timestamp$. If the latter condition is not met, it will simply return last_tvl.

This function returns `last_tvl[i]`, which represents the ema tvl of the pool. This variable is updated everytime when calling `price_w()` and $last_{timestamp} < block.timestamp$.
This value is subsequently used in the internal function `_raw_price()` to compute the weighted price of ETH.

The function only re-calculates the ema tvl when $last_{timestamp} < block.timestamp$, otherwise it will just return `last_tvl` again as it is still the same timestamp. 


### *Calculate Smoothing Factor (α)*
When calculating the smoothing factor $\alpha$, the formula is converted to an int256 because `exp()` requires an int256 as input.
When calculating the smoothing factor, represented as $\alpha$, the formula is converted to an int256 type because the exp() function requires an int256 as its input.

In [72]:
# last_tvl stored in last_tvl list. this is used if last_timestamp < block_timestamp
for i in range (0, N_POOLS):
    x = price_oracle_contract.last_tvl(i)
    last_tvl.append(x)


# calculates new last_tvl and stores it in last_tvl_new list.
if last_timestamp < block_timestamp:
    # alpha is a smoothing factory for the tvls
    alpha = float(math.exp(-(block_timestamp - last_timestamp) * 10**18 / TVL_MA_TIME))

    for i in range(0, N_POOLS):
        tri = ape.Contract(TRICRYPTO[i])
        tvl = tri.totalSupply() * tri.virtual_price() / 10**18
        last_tvl_calc = (tvl * (10**18 - alpha) + last_tvl[i] * alpha) / 10**18
        last_tvl_new.append(last_tvl_calc)

## **Calculate Raw Price**

This function requires `tvl` (which was calculated in the step above) and `agg_price` ((which essential is `STABLESWAP_AGGREGATOR.price()`) as input to calculate the raw price of the collateral.
`_raw_price()` calculates the raw price of the collateral. The function requires the inputs `tvls` (from `_ema_tvl()`) and `agg_price` (from `STABLESWAP_AGGREGATOR.price()`).

In [73]:
weighted_price = 0
weights = 0


for i in range(0, N_POOLS):
    tri = ape.Contract(TRICRYPTO[i])
    stable = ape.Contract(STABLESWAP[i])
    p_crypto_r = tri.price_oracle(1)
    p_stable_r = stable.price_oracle()
    p_stable_agg = stable_agg.price()
    if last_timestamp < block_timestamp:
        weight = last_tvl_new[i]
    else:
        weight = last_tvl[i]
    weights += weight

    weighted_price += p_crypto_r * p_stable_agg / p_stable_r * weight

p_eth = (weighted_price / weights) / 10**18
print(p_eth)

1716.9376650854756


Next, the price of stETH w.r.t ETH is capped. It's determined by taking the lesser value between the price of stETH in the curve pool and 1. This adjustment is necessary because if the stETH price in the pool exceeds 1, it creates an arbitrage opportunity. Traders could convert ETH for stETH at a 1:1 ratio and then sell it in the pool, which should push the exchange rate back down to 1.

This limited value is then multiplied by `WSTETH.stEthPerToken()` which is the ratio of wsteth and steth.
This capped value is then multiplied by `WSTETH.stEthPerToken()`, which represents the ratio between wstETH and stETH.

Final step: the limited value of p_staked is then multiplied by the price of eth calculated above and then divided by the number of decimals ($10^{18}$).
In the final step, the obtained value of `p_staked` is multiplied by the weighted price of ETH, as calculated earlier, and then divided by the number of decimals (represented by $10^{18}$.

In [74]:
p_staked = STAKEDSWAP.price_oracle()

p_staked = min(p_staked, 10**18) * WSTETH.stEthPerToken() / 10**18

p_final = p_staked * p_eth / 10**18

In [75]:
print("--------")
print("FINAL WSTETH PRICE:")
print(p_final)
print("--------")

--------
FINAL WSTETH PRICE:
1953.2656279080861
--------


## **Chainlink Limits**

The oracle contracts have the possibility to make use of chainlink prices which act as saftey limits. When toggled on, these limitations essentially hit when the chainlink price deviates more than 1.5% (`BOUND_SIZE`) from the internal price oracles.
The oracle contracts have the option to utilize Chainlink prices, which serve as safety limits. When enabled, these limits are triggered if the Chainlink price deviates by more than 1.5% (represented by `BOUND_SIZE`) from the internal price oracles.

Chainlink limits can be turned on and off by calling `set_use_chainlink(do_it: bool)`, which can only be done by the admin of the factory contract.

In [76]:
# chainlink limits


In [77]:
# clear last_tvl_new list as its not needed anymore and needs to be empty to not fuck up the calculation when calling again
last_tvl_new.clear()